# get service annotations from k8s api

In [2]:
from kubernetes import client, config

In [17]:
# Configs can be set in Configuration class directly or using helper utility
config.load_kube_config(context="k8s-houzz.stghouzz.com")

In [64]:
v1 = client.CoreV1Api()
print("expected lb binding k8s svc")
print("svc name / external-dns.hostname")
ret = v1.list_service_for_all_namespaces(watch=False)
kvpair = {}
for i in ret.items:
    if  i.metadata.annotations != None and 'external-dns.alpha.kubernetes.io/hostname' in i.metadata.annotations.keys():
        print("{} / {} / {}".format(i.metadata.name, i.metadata.annotations['external-dns.alpha.kubernetes.io/hostname'], i.status.load_balancer.ingress[0].hostname))
        kvpair[i.metadata.name] = i.metadata.annotations['external-dns.alpha.kubernetes.io/hostname']
        

expected lb binding k8s svc
svc name / external-dns.hostname
istio-ingressgateway-c2-thrift-main / c2-thrift-main.houzz1.k8s-houzz.stghouzz.com / internal-a0046c9a4eef84daa936fe89bae1d847-642646328.us-west-2.elb.amazonaws.com
istio-ingressgateway-c2-web-main / c2-web-main.houzz1.k8s-houzz.stghouzz.com / internal-afc9584065d9d4cc69234464dbd70b05-694029955.us-west-2.elb.amazonaws.com
istio-ingressgateway-jukwaa-main / jukwaa-main.houzz1.k8s-houzz.stghouzz.com / internal-a43e3cb1bf046403cbc5829326c14ed2-1954115257.us-west-2.elb.amazonaws.com
istio-ingressgateway-c2-thrift-php7 / c2-thrift-main.houzz2.k8s-houzz.stghouzz.com / internal-af8c3b77981fe4195bf0c60aa0741f68-1563532737.us-west-2.elb.amazonaws.com
istio-ingressgateway-c2-web-php7 / c2-web-main.houzz2.k8s-houzz.stghouzz.com / internal-ab0de2c46fd9e430b9488dd6f057d280-560239091.us-west-2.elb.amazonaws.com
istio-ingressgateway-internal-tools / internal-tools.houzz2.k8s-houzz.stghouzz.com / internal-a3786f0432ff049b7a9ae0dd2068fe55-516

In [26]:
print(kvpair)

{'istio-ingressgateway-c2-thrift-main': 'c2-thrift-main.houzz3.k8s-houzz.stghouzz.com', 'istio-ingressgateway-c2-web-main': 'c2-web-main.houzz3.k8s-houzz.stghouzz.com', 'istio-ingressgateway-jukwaa-main': 'jukwaa-main.stghouzz.k8s-houzz.stghouzz.com', 'istio-ingressgateway-c2-thrift-php7': 'c2-thrift-main.stghouzz.k8s-houzz.stghouzz.com', 'istio-ingressgateway-c2-web-php7': 'c2-web-main.stghouzz.k8s-houzz.stghouzz.com', 'istio-ingressgateway-internal-tools': 'internal-tools.stghouzz.k8s-houzz.stghouzz.com', 'istio-ingressgateway-next-pro-site-main': 'next-pro-site-main.stghouzz.k8s-houzz.stghouzz.com', 'istio-ingressgateway-cdp-api': 'cdp-api.stghouzz.k8s-houzz.stghouzz.com', 'istio-ingressgateway-jukwaa-graphql': 'jukwaa-graphql.stghouzz.k8s-houzz.stghouzz.com', 'istio-ingressgateway-jukwaa-v2': 'jukwaa-v2.stghouzz.k8s-houzz.stghouzz.com'}


# get record from specific route53 zone

In [27]:
import boto3

session = boto3.Session(profile_name='staging')

In [29]:
route53_client = session.client('route53')

In [31]:
stg_all_zone = route53_client.list_hosted_zones()

In [36]:
all_zone_records = route53_client.list_resource_record_sets(
            HostedZoneId='/hostedzone/Z0323705FCDT3CK76BMG'
        )['ResourceRecordSets']

In [63]:
all_expeted_domains = list(kvpair.values())

print("actual route53 lb binding")

for domain in all_expeted_domains:
    for rr in all_zone_records:
        ok_flag = False
        if rr['Name'][:-1] == domain:
            if rr['Type'] == 'A':
                print(domain, rr['AliasTarget']['DNSName'], rr['AliasTarget']['EvaluateTargetHealth'])

actual route53 lb binding
c2-thrift-main.houzz3.k8s-houzz.stghouzz.com internal-abb6307fce8694407b7d1e1d70a39e7e-2010731691.us-west-2.elb.amazonaws.com. True
c2-web-main.houzz3.k8s-houzz.stghouzz.com internal-a2133a306383d4489a05a0083e2b72c1-869519761.us-west-2.elb.amazonaws.com. True
jukwaa-main.stghouzz.k8s-houzz.stghouzz.com internal-a2f8f802a1a844287b7d61dadd0f5efc-141810882.us-west-2.elb.amazonaws.com. True
c2-thrift-main.stghouzz.k8s-houzz.stghouzz.com internal-abb9c51c1d29941c297e373b8aee67c4-1111716126.us-west-2.elb.amazonaws.com. True
c2-web-main.stghouzz.k8s-houzz.stghouzz.com internal-a0401bacdaa76469db857323d8b9ad43-2029665750.us-west-2.elb.amazonaws.com. True
internal-tools.stghouzz.k8s-houzz.stghouzz.com internal-afa4f98024c1644988c10fef5236fb0c-1177002133.us-west-2.elb.amazonaws.com. True
next-pro-site-main.stghouzz.k8s-houzz.stghouzz.com internal-a8cc922849f674fc990eb9d2b1305c44-1667914448.us-west-2.elb.amazonaws.com. True
cdp-api.stghouzz.k8s-houzz.stghouzz.com interna

In [44]:
print(all_expeted_domains)

['c2-thrift-main.houzz3.k8s-houzz.stghouzz.com', 'c2-web-main.houzz3.k8s-houzz.stghouzz.com', 'jukwaa-main.stghouzz.k8s-houzz.stghouzz.com', 'c2-thrift-main.stghouzz.k8s-houzz.stghouzz.com', 'c2-web-main.stghouzz.k8s-houzz.stghouzz.com', 'internal-tools.stghouzz.k8s-houzz.stghouzz.com', 'next-pro-site-main.stghouzz.k8s-houzz.stghouzz.com', 'cdp-api.stghouzz.k8s-houzz.stghouzz.com', 'jukwaa-graphql.stghouzz.k8s-houzz.stghouzz.com', 'jukwaa-v2.stghouzz.k8s-houzz.stghouzz.com']


# send metrics to prom's push gateway

pushgateway runs on  separate port, by default 9091.  we have open the port on LB just for this script if use push


In [2]:
from prometheus_client import CollectorRegistry, Counter, push_to_gateway


In [3]:
registry = CollectorRegistry()

c = Counter('kailun_test_metric_a', 'Kailun Test Metric', ['label1', 'label2'])
c.labels('apple', 'pear').inc()
c.labels('orange', 'peach').inc()

In [4]:
# push_to_gateway('localhost:9091', job='batchA', registry=registry)